# Import and sandbox

In [ ]:
import numpy as np
import pandas as pd
import torch 
from tqdm import tqdm
import matplotlib.pyplot as plt


# check n_wells 

## Imports

In [ ]:
df_phenom = pd.read_parquet('/projects/synsight/data/openphenom/raw_well_embeddings/wells_em_openphenom.parquet')

In [ ]:
df_dinov2_s = pd.read_parquet('/projects/synsight/data/dinov2_small/raw_well_embeddings/wells_em_dinov2s.parquet')

In [ ]:
df_dinov2_g = pd.read_parquet('/projects/synsight/data/dinov2/raw_well_embeddings/wells_em_dinov2.parquet')

In [ ]:
df_resnet = pd.read_parquet('/projects/cpjump1/jump/images_embeddings/resnet50_raw/metadata/load_wells/df_meta_raw_well.parquet')

In [ ]:
df_chada = pd.read_parquet('/projects/synsight/data/chada/well_embeddings/metadata_raw_mean.parquet')

In [ ]:
df_chada_2 = pd.read_parquet('/projects/synsight/data/chada/chada_jump.parquet')

In [ ]:
df_meta = pd.read_csv('/projects/cpjump1/jump/metadata/complete_metadata.csv')

## Reference

In [ ]:
images_df_path = "/projects/cpjump1/jump/load_data/final"
images_df = pd.read_parquet(images_df_path)

images_df["Metadata_Batch"] = images_df["Metadata_Batch"].astype(str)
df_meta["Metadata_Batch"] = df_meta["Metadata_Batch"].astype(str)
images_df["Metadata_Plate"] = images_df["Metadata_Plate"].astype(str)
df_meta["Metadata_Plate"] = df_meta["Metadata_Plate"].astype(str)
images_df["Metadata_Source"] = images_df["Metadata_Source"].astype(str)
df_meta["Metadata_Source"] = df_meta["Metadata_Source"].astype(str)
images_df["Metadata_Well"] = images_df["Metadata_Well"].astype(str)
df_meta["Metadata_Well"] = df_meta["Metadata_Well"].astype(str)

merged_df = pd.merge(images_df, df_meta, on=["Metadata_Source", "Metadata_Plate", "Metadata_Well"])
merged_df

In [ ]:
len(merged_df)

In [ ]:
merged_df[(merged_df['Metadata_PlateType']=='COMPOUND')]["Metadata_JCP2022"].nunique()

In [ ]:
df_meta[(df_meta['Metadata_PlateType']=='COMPOUND')]["Metadata_JCP2022"].nunique()

In [ ]:
key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

df1 = merged_df[merged_df['Metadata_PlateType']=='COMPOUND'][['Metadata_Source', 'Metadata_Well', 'Metadata_Plate', 'Metadata_JCP2022']]
df2 = df_meta[(df_meta['Metadata_PlateType']=='COMPOUND')]
only_in_df1 = df1.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(df1, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

In [ ]:
only_in_df2[only_in_df2['Metadata_Source']!='source_7']['Metadata_Plate'].unique()

In [ ]:
key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

df1 = merged_df[merged_df['Metadata_PlateType']!='COMPOUND'][['Metadata_Source', 'Metadata_Well', 'Metadata_Plate', 'Metadata_JCP2022']]
df2 = df_meta[(df_meta['Metadata_PlateType'].isin(['TARGET2', 'CRISPR', 'ORF', 'TARGET1']))]
only_in_df1 = df1.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(df1, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

In [ ]:
only_in_df2['Metadata_Source'].value_counts()

In [ ]:
df = merged_df[merged_df["Metadata_PlateType"]=='COMPOUND'][['Metadata_Source', 'Metadata_Batch_x', 'Metadata_Plate',
       'Metadata_Well', 'Metadata_JCP2022', 'Metadata_InChI']]
df.columns = ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_InChI']

In [ ]:
ref_compound_df = df[~df['Metadata_JCP2022'].isin(['JCP2022_999999', 'JCP2022_UNKNOWN'])].drop_duplicates()[~df['Metadata_JCP2022'].isin(['JCP2022_999999', 'JCP2022_UNKNOWN'])].reset_index().drop(columns='index')

In [ ]:
ref_compound_df

In [ ]:
ref_compound_df['Metadata_JCP2022'].value_counts()[:10]

In [ ]:
ref_compound_df.columns

In [ ]:
count = ref_compound_df['Metadata_JCP2022'].value_counts().values

plt.figure(figsize=(10, 6))  # Set figure size
plt.hist(count[10:], bins=100, log=False, color='skyblue', edgecolor='black', alpha=0.7)

# Add labels and title
plt.title('Histogram of Well by JCP2022 id (excluding controls)', fontsize=16)
plt.xlabel('Count', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Add grid for better readability
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Count the number of unique plates per batch
plate_count_per_batch = ref_compound_df.groupby('Metadata_Batch')['Metadata_Plate'].nunique()

# Plot histogram
plt.figure(figsize=(10, 6))  # Set figure size
plt.hist(plate_count_per_batch.values, bins=range(1, plate_count_per_batch.max() + 2), color='salmon', edgecolor='black', alpha=0.7)

# Add labels and title
plt.title('Histogram of Plate Count by Batch', fontsize=16)
plt.xlabel('Number of Plates', fontsize=14)
plt.ylabel('Frequency of Batches', fontsize=14)

# Add grid for better readability
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Exemple : Si ref_compound_df est votre DataFrame
# Calculer les plaques uniques par batch
unique_plates_per_batch = ref_compound_df.groupby('Metadata_Batch')['Metadata_Plate'].nunique()

# Si un batch a moins de 10 plaques, toutes sont prises, sinon on limite à 10
total_selected_plates = unique_plates_per_batch.apply(lambda x: min(x, 15)).sum()

total_selected_plates

In [ ]:
filtered_df.sample(n=3)['Metadata_Plate']

In [ ]:
n = 10

# Grouper par batch et sélectionner les n premières plaques uniques
filtered_df = (
    ref_compound_df.groupby('Metadata_Batch')
    .apply(lambda group: group[group['Metadata_Plate'].isin(group['Metadata_Plate'].unique()[:n])])
    .reset_index(drop=True)
)

# Afficher le DataFrame filtré
filtered_df

## Counts

### Resnet

In [ ]:
len(ref_compound_df)

In [ ]:
len(df_resnet)

In [ ]:

df2 = df_resnet.copy(deep=True)

key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

only_in_ref = ref_compound_df.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(ref_compound_df, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)



In [ ]:
print(len(only_in_df2))
print(len(only_in_ref))

### chada

In [ ]:
len(df_chada)

In [ ]:

df2 = df_chada.copy(deep=True)

key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

only_in_ref = ref_compound_df.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(ref_compound_df, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

print(len(only_in_df2))
print(len(only_in_ref))

In [ ]:
only_in_ref # One molecule that is strange in formula so should have failed while preprocessed by @ThomasLemercier

### dino (s and g have the exact same lines)

#### dino s

In [ ]:
len(df_dinov2_s.drop_duplicates(subset=key_columns))

In [ ]:
duplicates = df_dinov2_s.groupby(key_columns).filter(lambda x: len(x) > 1)


In [ ]:
embeddings = []
for path in duplicates['path_embedding']:
    embeddings.append(torch.load(path))
    

In [ ]:
(embeddings[3] == embeddings[5]).all()

In [ ]:
df_dinov2_s.drop(index=[186948, 734868, 811287], inplace=True)


In [ ]:
df2 = df_dinov2_s.copy(deep=True)
key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']
only_in_ref = ref_compound_df.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(ref_compound_df, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

print(len(only_in_df2))
print(len(only_in_ref))

In [ ]:
df_dinov2_s.columns

In [ ]:
df_dinov2_s_good = ref_compound_df.merge(df_dinov2_s[['Metadata_Well', 'Metadata_Source', 'Metadata_Plate', 'Metadata_Is_dmso','Metadata_Row', 'Metadata_Col', 'path_embedding']], on=key_columns, how='inner')
df_dinov2_s_good.reset_index(drop=True, inplace=True)

In [ ]:
df_dinov2_s_good

In [ ]:
df_dinov2_s_good.to_parquet('/projects/synsight/data/dinov2_small/raw_well_embeddings/compounds_wells_em_dinov2s.parquet', index=False)

In [ ]:
df_dinov2_s_good = pd.read_parquet('/projects/synsight/data/dinov2_small/raw_well_embeddings/compounds_wells_em_dinov2s.parquet')

In [ ]:
df_dinov2_s_good

#### dino g

In [ ]:
duplicates = df_dinov2_g.groupby(key_columns).filter(lambda x: len(x) > 1)
duplicates

In [ ]:
df_dinov2_g.drop(index=[25526, 167155, 444675], inplace=True)

In [ ]:
df_dinov2_g_good = ref_compound_df.merge(df_dinov2_g[['Metadata_Well', 'Metadata_Source', 'Metadata_Plate', 'Metadata_Is_dmso','Metadata_Row', 'Metadata_Col', 'path_embedding']], on=key_columns, how='inner')
df_dinov2_g_good.reset_index(drop=True, inplace=True)

In [ ]:
df_dinov2_g_good['Metadata_InChI']

In [ ]:
df_dinov2_g_good[df_dinov2_g_good['Metadata_JCP2022']=='JCP2022_002781']['Metadata_InChI'].iloc[0]

In [ ]:
df_dinov2_g_good['Metadata_JCP2022'].value_counts()

In [ ]:
df_dinov2_g_good.to_parquet('/projects/synsight/data/dinov2/raw_well_embeddings/compounds_wells_em_dinov2g.parquet', index=False)

### OpenPhenom

In [ ]:
len(df_phenom)

In [ ]:

df2 = df_phenom.copy(deep=True)

key_columns = ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well']

only_in_ref = ref_compound_df.merge(df2, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
only_in_df2 = df2.merge(ref_compound_df, on=key_columns, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

print(len(only_in_df2))
print(len(only_in_ref))

In [ ]:
df_phenom_good = ref_compound_df.merge(df_phenom[['Metadata_Well', 'Metadata_Source', 'Metadata_Plate', 'Metadata_Is_dmso','Metadata_Row', 'Metadata_Col', 'path_embedding']], on=key_columns, how='inner')
df_phenom_good.reset_index(drop=True, inplace=True)

In [ ]:
df_phenom_good

In [ ]:
df_phenom_good.to_parquet('/projects/synsight/data/openphenom/raw_well_embeddings/compounds_wells_em_openphenom.parquet', index=False)

In [ ]:
df_phenom = pd.read_parquet('/projects/synsight/data/openphenom/raw_well_embeddings/wells_em_openphenom.parquet')